# Лабораторная работа №6


_ПИН-212 Жан-Лин Никита_

In [1]:
!pip install catboost

In [3]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler

In [5]:
data = pd.read_csv('smoking_driking_dataset_Ver03.csv', encoding='utf-8')
print(data.head(5))

    sex  age  height  weight  waistline  sight_left  sight_right  hear_left  \
0  Male   35     170      75       90.0         1.0          1.0        1.0   
1  Male   30     180      80       89.0         0.9          1.2        1.0   
2  Male   40     165      75       91.0         1.2          1.5        1.0   
3  Male   50     175      80       91.0         1.5          1.2        1.0   
4  Male   50     165      60       80.0         1.0          1.2        1.0   

   hear_right    SBP  ...  LDL_chole  triglyceride  hemoglobin  urine_protein  \
0         1.0  120.0  ...      126.0          92.0        17.1            1.0   
1         1.0  130.0  ...      148.0         121.0        15.8            1.0   
2         1.0  120.0  ...       74.0         104.0        15.8            1.0   
3         1.0  145.0  ...      104.0         106.0        17.6            1.0   
4         1.0  138.0  ...      117.0         104.0        13.8            1.0   

   serum_creatinine  SGOT_AST  SGOT_AL

In [7]:
# Преобразуем 'sex' в числовую категорию
data['sex'] = data['sex'].map({'Male': 1, 'Female': 0}).astype(int)

# Преобразуем 'DRK_YN' в бинарные метки
data['DRK_YN'] = data['DRK_YN'].map({'Y': 1, 'N': 0}).astype(int)

# Проверка уникальных значений в целевой переменной
print("Уникальные значения в data['DRK_YN']:", data['DRK_YN'].unique())

Уникальные значения в data['DRK_YN']: [1 0]


In [9]:
columns_to_drop = [
    'SMK_stat_type_cd',  # Если не нужен
    'sight_left', 'sight_right',  # Если зрение с обеих сторон не добавляет дополнительной информации
    'hear_left', 'hear_right',  # Если слух с обеих сторон не добавляет дополнительной информации
    'serum_creatinine', 'SGOT_AST', 'SGOT_ALT', 'gamma_GTP'  # Если лабораторные тесты дублируют информацию
]

# Удаление столбцов
data = data.drop(columns=columns_to_drop, axis=1)

# Проверка результата
print(data.head())

   sex  age  height  weight  waistline    SBP   DBP   BLDS  tot_chole  \
0    1   35     170      75       90.0  120.0  80.0   99.0      193.0   
1    1   30     180      80       89.0  130.0  82.0  106.0      228.0   
2    1   40     165      75       91.0  120.0  70.0   98.0      136.0   
3    1   50     175      80       91.0  145.0  87.0   95.0      201.0   
4    1   50     165      60       80.0  138.0  82.0  101.0      199.0   

   HDL_chole  LDL_chole  triglyceride  hemoglobin  urine_protein  DRK_YN  
0       48.0      126.0          92.0        17.1            1.0       1  
1       55.0      148.0         121.0        15.8            1.0       0  
2       41.0       74.0         104.0        15.8            1.0       0  
3       76.0      104.0         106.0        17.6            1.0       0  
4       61.0      117.0         104.0        13.8            1.0       0  


In [11]:
# Проверка уникальных значений в целевой переменной
print("Уникальные значения в data['DRK_YN']:", data['DRK_YN'].unique())

Уникальные значения в data['DRK_YN']: [1 0]


In [13]:
# Разделение данных на признаки и целевую переменную
X = data.drop('DRK_YN', axis=1)
Y = data['DRK_YN']

In [15]:
# Масштабирование числовых признаков
scaler = StandardScaler()
numeric_features = X.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
X[numeric_features] = scaler.fit_transform(X[numeric_features])

In [17]:
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

### Задание

In [21]:
# создаем модель
cat_boost_classifier = CatBoostClassifier()

In [23]:
cat_boost_params ={
    "iterations": range(1,6),
    'learning_rate': [0.1,0.25,0.5,0.75,1],
    'depth':range(1,6),
}

Определяем параметры, которые мы хотим оптимизировать для CatBoostClassifier.

iterations: этот ключ определяет количество итераций, которые CatBoost делает при обучении модели.

learning_rate: этот ключ определяет скорость обучения модели CatBoost.

depth: этот ключ определяет максимальную глубину деревьев в модели CatBoost.

In [26]:
# Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров
cat_boost_grid = GridSearchCV(cat_boost_classifier, cat_boost_params, cv=2, n_jobs=1)
cat_boost_grid.fit(X_train, Y_train)

0:	learn: 0.6791031	total: 161ms	remaining: 0us
0:	learn: 0.6797035	total: 1.02ms	remaining: 0us
0:	learn: 0.6607746	total: 745us	remaining: 0us
0:	learn: 0.6621591	total: 687us	remaining: 0us
0:	learn: 0.6375463	total: 663us	remaining: 0us
0:	learn: 0.6399395	total: 743us	remaining: 0us
0:	learn: 0.6232337	total: 625us	remaining: 0us
0:	learn: 0.6262634	total: 628us	remaining: 0us
0:	learn: 0.6172691	total: 768us	remaining: 0us
0:	learn: 0.6205769	total: 755us	remaining: 0us
0:	learn: 0.6791031	total: 679us	remaining: 679us
1:	learn: 0.6688272	total: 1.21ms	remaining: 0us
0:	learn: 0.6797035	total: 620us	remaining: 620us
1:	learn: 0.6705430	total: 1.2ms	remaining: 0us
0:	learn: 0.6607746	total: 616us	remaining: 616us
1:	learn: 0.6425091	total: 1.15ms	remaining: 0us
0:	learn: 0.6621591	total: 718us	remaining: 718us
1:	learn: 0.6461528	total: 1.41ms	remaining: 0us
0:	learn: 0.6375463	total: 662us	remaining: 662us
1:	learn: 0.6163186	total: 1.29ms	remaining: 0us
0:	learn: 0.6399395	total

GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostClassifier object at 0x0000019D237C3530>,
             n_jobs=1,
             param_grid={'depth': range(1, 6), 'iterations': range(1, 6),
                         'learning_rate': [0.1, 0.25, 0.5, 0.75, 1]})

In [31]:
best_cat_boost_model = CatBoostClassifier(**cat_boost_grid.best_params_)

In [35]:
best_cat_boost_model.fit(X_train, Y_train)

0:	learn: 0.6177123	total: 5.83ms	remaining: 17.5ms
1:	learn: 0.5889881	total: 9.5ms	remaining: 9.5ms
2:	learn: 0.5770969	total: 13.3ms	remaining: 4.42ms
3:	learn: 0.5693434	total: 17.1ms	remaining: 0us


In [37]:
cat_boost_predicted = best_cat_boost_model.predict(X_test)

In [41]:
print('Параметры:', cat_boost_grid.best_params_)
print('Оценка:\n', metrics.classification_report(Y_test, cat_boost_predicted, digits=5))

Параметры: {'depth': 5, 'iterations': 4, 'learning_rate': 0.5}
Оценка:
               precision    recall  f1-score   support

           0    0.71345   0.70230   0.70783      1216
           1    0.69909   0.71030   0.70465      1184

    accuracy                        0.70625      2400
   macro avg    0.70627   0.70630   0.70624      2400
weighted avg    0.70636   0.70625   0.70626      2400

